In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from datetime import datetime, timedelta
%matplotlib inline

In [13]:
#source_master = Path("../Resources/choice_df.csv")

source_tsla = Path("Resources/master_tesla.csv")

#master_df = pd.read_csv(source_master)
tsla_df = pd.read_csv(source_tsla)


In [14]:
tsla_df['ExpirationDate'] =  pd.to_datetime(tsla_df['ExpirationDate'], infer_datetime_format=True)
tsla_df['DataDate'] =  pd.to_datetime(tsla_df['DataDate'], infer_datetime_format=True)
tsla_df['time delta'] = tsla_df['ExpirationDate'] - tsla_df['DataDate']
tsla_df['% out'] = tsla_df['StrikePrice'] / tsla_df['UnderlyingPrice'] - 1

tsla_df.tail()

,Unnamed: 0,OptionKey,Symbol,ExpirationDate,AskPrice,AskSize,BidPrice,BidSize,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out
156907,805968,TSLA2021-06-18P25.002019-06-19,TSLA,2021-06-18,3.50,5,2.70,3,3.05,put,25.0,5,3,226.43,2019-06-19,730 days,-0.889591
156908,805972,TSLA2021-06-18P280.002019-06-19,TSLA,2021-06-18,105.30,50,97.00,91,101.00,put,280.0,3,8,226.43,2019-06-19,730 days,0.236585
156909,805974,TSLA2021-06-18P30.002019-06-19,TSLA,2021-06-18,5.40,15,2.42,15,3.85,put,30.0,7,17,226.43,2019-06-19,730 days,-0.867509
156910,805983,TSLA2021-06-18P5.002019-06-19,TSLA,2021-06-18,0.45,10,0.38,3,0.42,put,5.0,145,354,226.43,2019-06-19,730 days,-0.977918
156911,805988,TSLA2021-06-18P70.002019-06-19,TSLA,2021-06-18,13.45,67,11.00,16,11.35,put,70.0,20,2,226.43,2019-06-19,730 days,-0.690854


In [15]:
df = pd.DataFrame(tsla_df)
df = df.set_index(pd.to_datetime(df.DataDate, infer_datetime_format=True))
df['OptionKey'] = df['OptionKey'].map(lambda x:str(x)[:-10])
df = df.drop(columns = ['Unnamed: 0', 'AskSize', 'BidSize'])

df.head()

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out
DataDate,,,,,,,,,,,,,,
2019-06-20,TSLA2019-08-02C180.00,TSLA,2019-08-02,48.10,44.55,44.72,call,180.0,1,4,219.62,2019-06-20,43 days,-0.180403
2019-06-20,TSLA2019-08-02C187.50,TSLA,2019-08-02,42.00,39.05,42.92,call,187.5,10,0,219.62,2019-06-20,43 days,-0.146253
2019-06-20,TSLA2019-08-02C192.50,TSLA,2019-08-02,37.40,35.30,40.35,call,192.5,1,1,219.62,2019-06-20,43 days,-0.123486
2019-06-20,TSLA2019-08-02C200.00,TSLA,2019-08-02,32.00,31.25,36.80,call,200.0,2,5,219.62,2019-06-20,43 days,-0.089336
2019-06-20,TSLA2019-08-02C205.00,TSLA,2019-08-02,28.75,27.90,28.45,call,205.0,1,21,219.62,2019-06-20,43 days,-0.066570


In [17]:
#track PnL code- filter for only puts, slice dataframe length and sort
df = df[(df['ExpirationDate'] <= '2019-12-30') & (df.PutCall == 'put')]
df = df.sort_values(['OptionKey','time delta'])
                                                   
df.head()

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out
DataDate,,,,,,,,,,,,,,
2019-06-06,TSLA2019-06-07P115.00,TSLA,2019-06-07,0.01,0.0,0.01,put,115.0,3,88,205.95,2019-06-06,1 days,-0.441612
2019-06-06,TSLA2019-06-07P122.00,TSLA,2019-06-07,0.01,0.0,0.02,put,122.0,1,180,205.95,2019-06-06,1 days,-0.407623
2019-06-07,TSLA2019-06-07P125.00,TSLA,2019-06-07,0.01,0.0,0.01,put,125.0,1,910,204.50,2019-06-07,0 days,-0.388753
2019-06-06,TSLA2019-06-07P125.00,TSLA,2019-06-07,0.01,0.0,0.01,put,125.0,125,930,205.95,2019-06-06,1 days,-0.393057
2019-06-06,TSLA2019-06-07P128.00,TSLA,2019-06-07,0.01,0.0,0.01,put,128.0,30,221,205.95,2019-06-06,1 days,-0.378490


In [18]:
# load LSTM and RandomForest df
lstm_signals_df = pd.read_pickle('Resources/TSLA_LSTM_signals_df.pickle')
rf_signals_df = pd.read_pickle('Resources/TSLA_RF_signals_df.pickle')

In [19]:
merged_lstm = df.merge(lstm_signals_df, left_index = True, right_index = True, how= 'inner')

In [20]:
merged = merged_lstm.merge(rf_signals_df, left_index =True, right_index =True, how = 'inner')

In [21]:
merged.head()

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out,LSTM Predicted Signal,RF Predicted Value
2019-11-13,TSLA2019-11-15P155.00,TSLA,2019-11-15,0.05,0.0,0.00,put,155.0,3,1524,346.11,2019-11-13,2 days,-0.552165,1,1
2019-11-13,TSLA2019-11-15P165.00,TSLA,2019-11-15,0.01,0.0,0.00,put,165.0,91,0,346.11,2019-11-13,2 days,-0.523273,1,1
2019-11-13,TSLA2019-11-15P185.00,TSLA,2019-11-15,0.03,0.0,0.02,put,185.0,4,0,346.11,2019-11-13,2 days,-0.465488,1,1
2019-11-13,TSLA2019-11-15P190.00,TSLA,2019-11-15,0.02,0.0,0.00,put,190.0,1,0,346.11,2019-11-13,2 days,-0.451042,1,1
2019-11-13,TSLA2019-11-15P205.00,TSLA,2019-11-15,0.03,0.0,0.00,put,205.0,2,0,346.11,2019-11-13,2 days,-0.407703,1,1


In [23]:
#filter for OTM to make only entries to sell puts - not necessary

merged['LSTM OTM Cr Entry'] = np.where((merged['LSTM Predicted Signal'] ==1) &(merged['% out']> 0) & (merged['time delta'].map(lambda x:str(x).split(' ',1)[0]).astype(int) >= 7), merged.AskPrice*100, 0)
merged['RF OTM Cr Entry'] = np.where((merged['RF Predicted Value'] ==1) &(merged['% out']> 0) & (merged['time delta'].map(lambda x:str(x).split(' ',1)[0]).astype(int) >= 7), merged.AskPrice*100, 0)
merged['Bid Price'] = merged.BidPrice*100
merged = merged.sort_values(['OptionKey','DataDate'], ascending = True)

merged

,OptionKey,Symbol,ExpirationDate,AskPrice,BidPrice,LastPrice,PutCall,StrikePrice,Volume,OpenInterest,UnderlyingPrice,DataDate,time delta,% out,LSTM Predicted Signal,RF Predicted Value,LSTM OTM Cr Entry,RF OTM Cr Entry,Bid Price
2019-11-14,TSLA2019-11-15P135.00,TSLA,2019-11-15,0.01,0.00,0.01,put,135.0,1,234,349.35,2019-11-14,1 days,-0.613568,1,1,0.0,0.0,0.0
2019-11-13,TSLA2019-11-15P155.00,TSLA,2019-11-15,0.05,0.00,0.00,put,155.0,3,1524,346.11,2019-11-13,2 days,-0.552165,1,1,0.0,0.0,0.0
2019-11-13,TSLA2019-11-15P165.00,TSLA,2019-11-15,0.01,0.00,0.00,put,165.0,91,0,346.11,2019-11-13,2 days,-0.523273,1,1,0.0,0.0,0.0
2019-11-15,TSLA2019-11-15P165.00,TSLA,2019-11-15,0.01,0.00,0.01,put,165.0,1,904,352.17,2019-11-15,0 days,-0.531476,1,1,0.0,0.0,0.0
2019-11-15,TSLA2019-11-15P170.00,TSLA,2019-11-15,0.01,0.00,0.01,put,170.0,27,1890,352.17,2019-11-15,0 days,-0.517279,1,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-19,TSLA2019-12-27P475.00,TSLA,2019-12-27,118.35,114.00,121.84,put,475.0,10,0,359.52,2019-11-19,38 days,0.321206,-1,1,0.0,11835.0,11400.0
2019-11-22,TSLA2019-12-27P475.00,TSLA,2019-12-27,143.75,139.75,143.85,put,475.0,1,10,333.04,2019-11-22,35 days,0.426255,-1,1,0.0,14375.0,13975.0
2019-11-25,TSLA2019-12-27P475.00,TSLA,2019-12-27,140.50,136.35,134.80,put,475.0,3,4,336.34,2019-11-25,32 days,0.412261,-1,0,0.0,0.0,13635.0
2019-12-18,TSLA2019-12-27P490.00,TSLA,2019-12-27,97.60,96.30,98.85,put,490.0,7,0,393.15,2019-12-18,9 days,0.246344,1,1,9760.0,9760.0,9630.0
